# The Meme Machine

### A short script to make dank meme compilations

Currently the architecture works in multiple steps
1. Create reddit instance in praw
2. Use algo to pick out post to make dank meme video
3. Grab the top comments of post
4. Use tts to make it sing
5. Create a table of your tts and get seconds for each one
6. Splice it up with buffers
7. Put it together into one audio
8. Grab photos of posts
9. Use photos as frames for movie
10. Add backing track
11. Overlay frmes with sound time
12. Create thumbnail, outro and intro in web
13. Upload to YT with api
14. Reap the rewards

In [1]:
import praw
import pandas as pd
import datetime as dt
import unicodedata
import numpy as np

BASE_TRANSITION = 0
SPLIT_TRANSITION = 1
REPLY_TRANSITION = 2

In [100]:
reddit = praw.Reddit(client_id='GO5OTc3JcEHijg',
                    client_secret='4hYC9dphKz6ZgjlXnF8OvUl912c',
                    user_agent='ece188project',
                    username='turtle673',
                    password='wizard')

In [101]:
post = reddit.submission(id='bekeb1')
post.comments.replace_more(limit=0);

In [437]:
askReddit = reddit.subreddit('askreddit')

title = post.title
titleScore = post.score
titleAuthor = '[deleted]' if post.author is None else post.author.name
try:
    titleSilver = post.gildings['gid_1']
except:
    titleSilver = 0
try:
    titleGold = post.gildings['gid_2']
except:
    titleGold = 0
try:
    titlePlat = post.gildings['gid_3']
except:
    titlePlat = 0

comments = post.comments
    
sanitizedTitle = unicodedata.normalize('NFKD', title).encode('ascii', 'ignore')

In [438]:
print("There are " + str(len(comments)) + " total comments")

There are 91 total comments


In [439]:
def commentValue(comment):
    c = len(comment.body)
    charScore = c * 20 if c > 500 and c < 1000 else c * 10 if c > 1000 and c < 1500 else -1000000 if c > 2000 else c
    upvoteScore = comment.score * 0.5 if comment.score > 100 else -1000000
    gildScore = (comment.silver * 1000) + (comment.gold * 5000) + (comment.platinum * 20000)
    if comment.reply is not None:
        replyScore = commentValue(comment.reply)
    else:
        replyScore = 0
    return int(charScore + upvoteScore + gildScore + replyScore)

In [440]:
# holds all the info relating to rendering a single comment
# in the reply field it is a pointer to the top reply of that comment
# None if no eligible comment

class CommentInfo:
    def __init__(self, score, author, silver, gold, plat, html, commonBody):
        self.score = score
        self.author = author
        self.silver = silver
        self.gold = gold
        self.platinum = plat
        self.html_body = html
        self.body = commonBody
        self.reply = None


In [441]:
import re

def sanitizeString(comment):  
    sanitizedComment = unicodedata.normalize('NFKD', comment).encode('ascii', 'ignore') + " "
    s = re.sub(r'[\[\(]http.+?(?=[\)\]]).', '', sanitizedComment)
    s2 = re.sub(r'http.+?(?=[ \n])', '', s)
    s3 = re.sub(r'^[\[>]', '', s2, flags=re.M)
    s4 = re.sub(r'(\n(\n{1,}))', '\n', s3)
    return s4.replace("&#x200B;", "")

def convertToHTML(body):
    html = "<p>" + body.replace('\n', '</p>\n<p>').replace('<p></p>','').replace('<p> </p>','') + "</p>"
    return html

In [442]:
topComments= []
commentInfoValues = []

for comment in comments:
      
    sanitizedComment = sanitizeString(comment.body)
    htmlComment = convertToHTML(sanitizedComment)
    if htmlComment is None:
        print("--------------ERROR-------------------")
        print(sanitizedComment)
    
    gild = comment.gildings
    try:
        numSilver = gild['gid_1']
    except:
        numSilver = 0
    try:
        numGold = gild['gid_2']
    except:
        numGold = 0
    try:
        numPlat = gild['gid_3']
    except:
        numPlat = 0
    name = 'anonymous' if comment.author is None else comment.author.name
    
    base = CommentInfo(comment.score, name, numSilver, numGold, numPlat, htmlComment, sanitizedComment)
    value = commentValue(base)
    
    if value < 0:
        continue
    
    # get top reply
    replyInfo = []
    for c in comment.replies:
        replyScore = c.score
        replyAuthor = 'anonymous' if c.author is None else c.author.name
        try:
            replySilver = c.gildings['gid_1']
        except:
            replySilver = 0
        try:
            replyGold = c.gildings['gid_2']
        except:
            replyGold = 0
        try:
            replyPlat = c.gildings['gid_3']
        except:
            replyPlat = 0
        sanitizedReply = sanitizeString(c.body)
        htmlReply = convertToHTML(sanitizedReply)
        
        #reply1 = CommentInfo(replyScore, replyAuthor, replySilver, replyGold, replyPlat, c.body_html.encode("utf8").replace("\xe2\x80\x99", "'").replace("&amp;", "and").replace("&#39;", "'"), sanitizedReply)
        reply1 = CommentInfo(replyScore, replyAuthor, replySilver, replyGold, replyPlat, htmlReply, sanitizedReply)
        replyValue = commentValue(reply1)
        replyInfo.append((replyValue, reply1, c))
    
    replyInfo.sort(reverse=True)
    try:
        topReply = replyInfo[0]  
        # get top reply of top reply
        thirdInfo = []
        for third in topReply[2].replies:
            thirdScore = third.score
            thirdAuthor = 'anonymous' if third.author is None else third.author.name
            try:
                thirdSilver = third.gildings['gid_1']
            except:
                thirdSilver = 0
            try:
                thirdGold = third.gildings['gid_2']
            except:
                thirdGold = 0
            try:
                thirdPlat = third.gildings['gid_3']
            except:
                thirdPlat = 0
            sanitizedThird = sanitizeString(third.body)
            thirdHTML = convertToHTML(sanitizedThird)
            
            thirdI = CommentInfo(thirdScore, thirdAuthor, thirdSilver, thirdGold, thirdPlat, thirdHTML, sanitizedThird)
            thirdValue = commentValue(thirdI)
            thirdInfo.append((thirdValue, thirdI, third))

        thirdInfo.sort(reverse=True)
        try:
            topThirdReply = thirdInfo[0]
        except:
            print("Third level reply failed")
            topThirdReply = [-10000000]
    except:
        print("top Level reply failed")
        topReply = [-10000000]
  
    # creating reply chain
    scalar = 0.5
    if topReply[0] > value * scalar:
        base.reply = topReply[1]
        if topThirdReply[0] > topReply[0] * scalar:
            topReply[1].reply = topThirdReply[1]
    
    commentInfoValues.append((value, base))

Third level reply failed
top Level reply failed
top Level reply failed
Third level reply failed
Third level reply failed
Third level reply failed
Third level reply failed
Third level reply failed
top Level reply failed
Third level reply failed
Third level reply failed
Third level reply failed
top Level reply failed
Third level reply failed
Third level reply failed
Third level reply failed
top Level reply failed
Third level reply failed
Third level reply failed
top Level reply failed
Third level reply failed
top Level reply failed
top Level reply failed
Third level reply failed
top Level reply failed
top Level reply failed
top Level reply failed
top Level reply failed
Third level reply failed
top Level reply failed
top Level reply failed
top Level reply failed
Third level reply failed
Third level reply failed


In [443]:
# gaming more valuable comments
commentInfoValues.sort(reverse=True)
commentInfo = []
for i in range(len(commentInfoValues)):
    commentInfo.append(commentInfoValues[i][1])

print("There are " + str(len(commentInfo)) + " comments remaining")

There are 77 comments remaining


In [444]:
profanity = []
bannedWords = open("profanity.txt", "r")
for w in bannedWords:
    profanity.append(w.replace("\n", ""))
    
flaggedComments = []
for info in commentInfo:
    comment = info.body
    comment = comment.lower()
    unacceptable = 0
    for w in profanity:
        if w in comment:
            unacceptable += 1

    flaggedComments.append(unacceptable/float(len(comment)) >= 0.2)
    
flaggedComments = np.asarray(flaggedComments)
commentInfo = np.asarray(commentInfo)
acceptable = commentInfo[~flaggedComments]

In [445]:
len(commentInfo)

77

# Creating Images

### Get Comment Images

In [446]:
import cv2
import os
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
import urllib

videoSize = (1280, 720)

j = -1
#for i in range(len(commentInfo)):
i = 0
size = len(acceptable)

def renderComment(driver, comment, i, screenshot):
    fName = ""
    score = comment.score
    user = comment.author
    silver = comment.silver
    gold = comment.gold
    plat = comment.platinum
    body = comment.html_body
    url = "http://localhost:8000?score=" + str(score)\
            + "&user=" + user\
            + "&silver=" + str(silver)\
            + "&gold=" + str(gold)\
            + "&platinum=" + str(plat)\
            + "&comment=" + urllib.quote_plus(body)
    try:
        driver.get(url)
        if screenshot:
            fName = '/tmp/comment-' + str(i) + '-.png'
            screenshot = driver.save_screenshot(fName)
        return fName
    except:
        print("timeout exception fetching URL, failing")
        raise Exception("timeout exception")

    
def renderComments(comments, screenshot):
    #DRIVER = 'chromedriver'
    #driver = webdriver.Chrome(DRIVER)
    driver = webdriver.Firefox()
    driver.set_window_size(videoSize[0], videoSize[1])
    driver.set_page_load_timeout(5)
    success = []
    count = 0
    for c in comments:
        try:
            c.image = renderComment(driver, c.commentInfo, count, screenshot)
            success.append(True)
            count += 1
        except:
            success.append(False)
    driver.quit()
    return success

In [447]:
from HTMLParser import HTMLParser

class MLStripper(HTMLParser):
    def __init__(self):
        self.reset()
        self.fed = []
    def handle_data(self, d):
        self.fed.append(d)
    def get_data(self):
        return ''.join(self.fed)

def strip_tags(html):
    s = MLStripper()
    s.feed(html)
    return s.get_data()

In [448]:
def getCommentSplits(c):
    numMatching = 20
    groups = c.html_body.split("<p>")
    paras = []
    for p in groups:
        if len(p) > 0:
            paras.append("<p>" + p)

    sets = []
    s = ""
    count = 0
    for i in range(len(paras)):
        if (count % 3) == 0 or len(s) > 1500:
            sets.append(s)
            s = ""
            count = 0
        s = s + paras[i]
        count += 1
    sets = sets[1:]
    sets.append(s)
    
    indices = []

    for s in sets:
        try:
            saniKey = strip_tags(s)
            key = saniKey[:numMatching] if len(saniKey) > numMatching else saniKey[:10] if len(saniKey) > 10 else saniKey[:5]
            idx = c.body.index(key)
            indices.append(idx)
        except:
            print("Failed To Find Substring")
            print("-----------FULL_KEY----------------")
            print(repr(saniKey))
            print("-----------SanitizedHTML_KEY-------------")
            print(repr(key))
            print("-----------Actual_BODY---------------")
            print(repr(c.body))
            raise e
        
    bodyPairs = []
    for i in range(0, len(indices)):
        if i == 0:
            commonBody = c.body[:indices[1]] if len(indices) != 1 else c.body
            bodyPairs.append((commonBody, sets[i]))
            continue
        if i == len(indices)-1:
            commonBody = c.body[indices[i]:]
            bodyPairs.append((commonBody, sets[i]))
            continue
        before = indices[i]
        after = indices[i+1]
        commonBody = c.body[before:after]
        bodyPairs.append((commonBody, sets[i]))

    commentSplits = []
    for p in bodyPairs:
        commentSplits.append(CommentInfo(c.score, c.author, c.silver, c.gold, c.platinum, p[1], p[0]))
    return commentSplits

In [449]:
count = 0
class CommentClip:
    def __init__(self, commentInfo, transitionType):
        self.commentInfo = commentInfo
        self.transitionType = transitionType
        self.audio = None
        self.image = None
'''      
commentQueue = []
for cInfo in acceptable:
    commentQueue.append(CommentClip(cInfo, BASE_TRANSITION))
    if cInfo.reply is not None:
        commentQueue.append(CommentClip(cInfo.reply, REPLY_TRANSITION))
        if cInfo.reply.reply is not None:
            commentQueue.append(CommentClip(cInfo.reply.reply, REPLY_TRANSITION))
''' 
commentQueue = []
for cInfo in acceptable:
    splits = getCommentSplits(cInfo)
    commentQueue.append(CommentClip(splits[0], BASE_TRANSITION))
    if len(splits) != 1:
        for s in splits[1:]:
            commentQueue.append(CommentClip(s, SPLIT_TRANSITION))
    if cInfo.reply is not None:
        splits = getCommentSplits(cInfo.reply)
        commentQueue.append(CommentClip(splits[0], REPLY_TRANSITION))
        if len(splits) != 1:
            for s in splits[1:]:
                commentQueue.append(CommentClip(s, SPLIT_TRANSITION))
        if cInfo.reply.reply is not None:
            splits = getCommentSplits(cInfo.reply.reply)
            commentQueue.append(CommentClip(splits[0], REPLY_TRANSITION))
            if len(splits) != 1:
                for s in splits[1:]:
                    commentQueue.append(CommentClip(s, SPLIT_TRANSITION))



In [450]:
'''
c = acceptable[17]
groups = c.html_body.split("<p>")
paras = []
for p in groups:
    if len(p) > 0:
        paras.append("<p>" + p)

sets = []
s = ""
count = 0
for i in range(len(paras)):
    if (count % 3) == 0 or len(s) > 1500:
        sets.append(s)
        s = ""
        count = 0
    s = s + paras[i]
    count += 1
sets = sets[1:]
sets.append(s)

indices = []
for s in sets:
    try:
        saniKey = strip_tags(s)
        key = saniKey[:15] if len(saniKey.split('\n')[0]) > 15 else saniKey.split('\n')[0]
        idx = c.body.index(key)
        indices.append(idx)
    except:
        print("Failed To Find Substring")
        print("-----------SanitizedHTML_KEY-------------")
        print(repr(key))
        print("-----------Actual_BODY---------------")
        print(repr(c.body))
        raise e

for cI in getCommentSplits(acceptable[17]):
    print("----------------------")
    print(cI.body)
'''

'\nc = acceptable[17]\ngroups = c.html_body.split("<p>")\nparas = []\nfor p in groups:\n    if len(p) > 0:\n        paras.append("<p>" + p)\n\nsets = []\ns = ""\ncount = 0\nfor i in range(len(paras)):\n    if (count % 3) == 0 or len(s) > 1500:\n        sets.append(s)\n        s = ""\n        count = 0\n    s = s + paras[i]\n    count += 1\nsets = sets[1:]\nsets.append(s)\n\nindices = []\nfor s in sets:\n    try:\n        saniKey = strip_tags(s)\n        key = saniKey[:15] if len(saniKey.split(\'\n\')[0]) > 15 else saniKey.split(\'\n\')[0]\n        idx = c.body.index(key)\n        indices.append(idx)\n    except:\n        print("Failed To Find Substring")\n        print("-----------SanitizedHTML_KEY-------------")\n        print(repr(key))\n        print("-----------Actual_BODY---------------")\n        print(repr(c.body))\n        raise e\n\nfor cI in getCommentSplits(acceptable[17]):\n    print("----------------------")\n    print(cI.body)\n'

In [451]:
renderedCorrectly = renderComments(commentQueue, True)

#rendered = renderComments([CommentClip(CommentInfo(200,"me",0,0,0,"<p>HelloWorld<io1><lk12></span></p>", "junk"), False)],True)

In [452]:
commentQueue = np.asarray(commentQueue)
renderedCorrectly = np.asarray(renderedCorrectly)
'''
errors = np.where(renderedCorrectly == False)
for eIndex in errors:
    try:
        nextComment = commentQueue[eIndex+1]
        if nextComment.isReply:
            renderedCorrectly[eIndex+1] = False
            if commentQueue[eIndex+2].isReply
                renderedCorrectly[eIndex+2] = False
'''
        
commentQueue = commentQueue[np.asarray(renderedCorrectly)]

In [453]:
print("There are " + str(len(acceptable)) + " comments remaining")

There are 77 comments remaining


### Get Title and Outro Image

In [454]:
driver = webdriver.Firefox()
driver.set_window_size(videoSize[0], videoSize[1])
url = "http://localhost:8000/title.html?score=" + str(titleScore)\
        + "&user=" + titleAuthor\
        + "&silver=" + str(titleSilver)\
        + "&gold=" + str(titleGold)\
        + "&platinum=" + str(titlePlat)\
        + "&comment=" + urllib.quote_plus(sanitizedTitle)

driver.get(url)
screenshot = driver.save_screenshot('/tmp/title-slide.png')

url = "http://localhost:8000/outro.html"
driver.get(url)
screenshot = driver.save_screenshot('/tmp/outro-slide.png')

driver.quit()

### Given an array of sanitized top comments, get tts audio for each

In [455]:
from gtts import gTTS
from playsound import playsound
from pydub import AudioSegment
import scipy.io.wavfile as wav
import resampy

In [456]:
def createTTS(comment, i):
    fName = createWav(comment, i)
    changeRate(fName, 25500)
    rsName = resampleWav(fName, 44100)
    return rsName

def createWav(comment, i):
    tts = gTTS(comment, lang='en')
    fName = "theOneTrueMeme" + str(i) + ".mp3"
    tts.save("/tmp/" + fName)
    sound = AudioSegment.from_mp3("/tmp/" + fName)
    wavName = "theOneTrueMeme" + str(i) + ".wav"
    sound.export("/tmp/" + wavName, format='wav')
    return wavName

def changeRate(fName, rate):
    originalRate, data = wav.read("/tmp/" + fName)
    wav.write("/tmp/" + fName, rate, data)
    
def resampleWav(fName, newRate):
    rate, data = wav.read("/tmp/" + fName)
    rsData = resampy.resample(data, rate, newRate)
    rsName = "/tmp/rs" + fName
    wav.write(rsName, newRate, rsData)
    return rsName

In [457]:
createTTS(sanitizedTitle, 0)

# todo perform DFS on acceptable commentInfos, making TTS of each
# all commentInfos in acceptable can be rendered due to above
for i in range(len(commentQueue)):
    try:
        commentQueue[i].audio = createTTS(commentQueue[i].commentInfo.body, i+1)
    except:
        print(i)
        break

wav.write("/tmp/buffer.wav", 44100, np.zeros(44100, np.int16))

In [458]:
backingTrack = AudioSegment.from_mp3('bensound-summer.mp3')
backingTrack.export('/tmp/background.wav', format='wav')
endingTrack = AudioSegment.from_mp3('bensound-ukulele.mp3')
endingTrack.export('/tmp/ending.wav', format='wav')

<open file '/tmp/ending.wav', mode 'wb+' at 0x7f6894b948a0>

In [459]:
rate, data = wav.read("/tmp/rstheOneTrueMeme3.wav")
print(rate)

44100


In [460]:
backingRate, backingData = wav.read('/tmp/background.wav')
endingRate, endingData = wav.read('/tmp/ending.wav')
wavData = []
rate = 44100

#append title tts
rate, data = wav.read("/tmp/rstheOneTrueMeme0.wav")
wavData.append(data)

for i in range(len(commentQueue)):
    rate, data = wav.read(commentQueue[i].audio)
    wavData.append(data)
    
bRate, bufferData = wav.read("/tmp/buffer.wav")

In [461]:
wavTimes = np.array([len(data) / float(44100) for data in wavData])
framesPerCommentAt32fps = wavTimes * 32
framesPerCommentAt32fps = framesPerCommentAt32fps.astype(int)

# append outro slide time 10s
framesPerCommentAt32fps = np.append(framesPerCommentAt32fps, (32 * 10))
print(framesPerCommentAt32fps)

[ 259  370  346  284  487  120  144  114  112   49  104  279  531  724
  237  126   42  239  298  278  108 1421 1160  172  130  163   46  284
  308  317  307  207  472  368  346  470  362  466  389  488  492  589
  357  441 1053  449  186 1472  998 1188  516  534  987  312  314  205
  114   80  277  272   36  273  279  227  342   81  177  412  156  215
  173   55  169  385   59  820  122   49  122  887  741   70  286  531
  173  299  155  140  385  101   99  109  284   59  109  214  161  159
  226  305  252  114  359  606  137  395  275  105  401  370  328  130
   51   97  248   83  108  229  247   76   46  173  255  169  221   99
  331  146   54   59   24  320]


In [462]:
# limit it to over 32 * 60 * 10 = 19200 frames
sumFrames = 0
stopPoint = 0
while sumFrames < 19200:
    sumFrames += framesPerCommentAt32fps[stopPoint]
    if sumFrames > 19200 or stopPoint >= len(framesPerCommentAt32fps)-1:
        break
    stopPoint += 1

print("Stop Point is: " + str(stopPoint))

Stop Point is: 49


In [463]:
framesPerCommentAt32fps = framesPerCommentAt32fps[:stopPoint+1]

# append 10s outro
framesPerCommentAt32fps = np.append(framesPerCommentAt32fps, 320)

In [464]:
# intro slide
mid = int(len(bufferData)/2)
combined = np.concatenate([bufferData[:mid], wavData[0], bufferData[mid:]])

# body slides
for i in range(1, stopPoint+1):
    if i == stopPoint or i == len(wavData)-1:
        combined = np.concatenate([combined, wavData[i]])
        break
    else:
        combined = np.concatenate([combined, wavData[i], bufferData])
    
# ending slides
#combined = np.concatenate([combined, bufferData, bufferData, bufferData, bufferData, bufferData])

In [465]:
wav.write("/tmp/allComments.wav", 44100, combined)

In [466]:
# removing backing sound from introduction

introLength = len(wavData[0]) + len(bufferData)
introPause = np.zeros((introLength, 2), dtype=np.int16)
backingWithIntroPause = np.concatenate([introPause, backingData])

In [467]:
backingWithIntroPause.shape

(9990962, 2)

In [468]:
newTTS = np.divide(combined, 2)
stereoTTS = np.transpose(np.stack((newTTS, newTTS)))

backingExtended = backingWithIntroPause
while len(backingExtended) < len(stereoTTS):
    backingExtended = np.concatenate([backingExtended, backingData])
    
reducedBacking = np.divide(backingExtended, 16)
together = np.add(stereoTTS, reducedBacking[:len(stereoTTS)])
togetherWithEnding = np.concatenate([together, endingData[:(44100*10)]])

wav.write('/tmp/combinedBacking.wav', 44100, togetherWithEnding)

### Only create video that is 10 minutes long

In [469]:
len(commentQueue)

130

In [470]:
from scipy.misc import imsave
import imageio

folder = '/tmp'
commentQueue = commentQueue[:stopPoint]
frames = [(c.image, c.transitionType) for c in commentQueue]

In [471]:
for i in range(len(frames)):
    if(i == len(frames)-1):
        break;
        
    leaving = frames[i][0]
    entering = frames[i+1][0]
    
    tt = frames[i+1][1]
    
    im1 = cv2.imread(leaving)
    im2 = cv2.imread(entering)
    im1 = cv2.cvtColor(im1, cv2.COLOR_BGR2RGB)
    im2 = cv2.cvtColor(im2, cv2.COLOR_BGR2RGB)
    height, width, depth = im1.shape
    
    if tt == BASE_TRANSITION: # horizontal transition
        margin = [53,53,53] * np.ones((height, width/4, 1))
        margin = margin.astype(int)
        transitionFull = np.concatenate((im1, margin, im2), axis=1).astype('uint8')

        tH, tW, tD = transitionFull.shape
        speed = [0.1,0.2,0.3,0.4,0.55,0.7,0.85,1,1,1.15,1.3,1.45,1.6,1.7,1.8,1.9,
                1.9,1.8,1.7,1.6,1.45,1.3,1.15,1,1,0.85,0.7,0.55,0.4,0.3,0.2,0.1]
        for offset in range(32):
            origin = int(sum(speed[:offset+1]) * (tW - width) / 32)
            transitionFrame = transitionFull[:, origin:width+origin]
            imageio.imwrite("/tmp/transition" + str(i) + "-" + str(i+1) + "f" + str(offset) + ".png", transitionFrame)
    
    elif tt == REPLY_TRANSITION:  # transition up with previous poster sticky, finally pushes out of way at last frames
        margin = [53,53,53] * np.ones((height/4, width, 1))
        margin = margin.astype(int)
        transitionFull = np.concatenate((im1,margin,im2), axis=0).astype('uint8')
        
        tH, tW, tD = transitionFull.shape
        speed1 = [0.2,0.3,0.4,0.5,0.6, 0.7, 0.8 ,0.9,1.1,1.3,1.5,1.7,
                1.7,1.5,1.3,1.2,1.1,1,1,1,1,1,1,1]
        speed2 = [1,1,1,1,1,1,1,1]
        for offset in range(24):
            headerSize = int(videoSize[1] * 0.2)
            header = transitionFull[:headerSize, :]
        
            body = transitionFull[headerSize:, :]
            bH, bW, bD = body.shape
            
            origin = int(sum(speed1[:offset+1]) * (bH - height) / len(speed1))
            transitionFrame = body[origin:origin+height-headerSize, :]
            transitionWithHead = np.concatenate([header, transitionFrame], axis=0)
            imageio.imwrite("/tmp/transition" + str(i) + "-" + str(i+1) + "f" + str(offset) + ".png", transitionWithHead)
        for offset in range(24,32):
            header = transitionFull[:headerSize, :]
            finalFull = np.concatenate((header, im2), axis=0).astype('uint8')
            fH, fW, fD = finalFull.shape
            
            origin = int(sum(speed2[:offset-23]) * (fH - height) / len(speed2))
            transitionFrame2 = finalFull[origin:origin+height,:]
            imageio.imwrite("/tmp/transition" + str(i) + "-" + str(i+1) + "f" + str(offset) + ".png", transitionFrame2)

    elif tt == SPLIT_TRANSITION:  # perfect for extended comment
        margin = [53,53,53] * np.ones((height/4, width, 1))
        margin = margin.astype(int)
        
        headerSize = int(videoSize[1] * 0.2)
        body = im2[headerSize:,:]
        margin2 = [53,53,53] * np.ones((headerSize, width, 1))
        transitionFull = np.concatenate((im1,margin,body,margin2), axis=0).astype('uint8')
        
        tH, tW, tD = transitionFull.shape
        speed = [0.1,0.2,0.3,0.4,0.55,0.7,0.85,1,1,1.15,1.3,1.45,1.6,1.7,1.8,1.9,
                1.9,1.8,1.7,1.6,1.45,1.3,1.15,1,1,0.85,0.7,0.55,0.4,0.3,0.2,0.1]
        for offset in range(32):
            headerSize = int(videoSize[1] * 0.2)
            header = transitionFull[:headerSize, :]
        
            body = transitionFull[headerSize:, :]
            bH, bW, bD = body.shape
            
            origin = int(sum(speed[:offset+1]) * (bH - height) / 32)
            transitionFrame = body[origin:origin+height-headerSize, :]
            transitionWithHead = np.concatenate([header, transitionFrame], axis=0)
            imageio.imwrite("/tmp/transition" + str(i) + "-" + str(i+1) + "f" + str(offset) + ".png", transitionWithHead)

#### The audio has now been constructed

# Generating Movie

- uses OpenCV to generate movie from frames
- uses MoviePy to overlay audio onto movie

In [472]:
folder = '/tmp'
videoName = '/tmp/output.avi'
frame = cv2.imread(frames[0][0])
height, width, layers = frame.shape

In [473]:
print(len(commentQueue))
len(framesPerCommentAt32fps)

49


51

'<p>Lobster used to be considered a poor mans food. </p>\n<p>In the 17th and 18th centuries American colonists found an overwhelming amount of lobster on the shore in giant mounds (sometimes up to a mans knees) there was so much of it in fact that a child could go down to the shore and come home with buckets full of dinner. People became sick of it and started feeding it to their farm animals while Natives used it as fertilizer. </p>\n<p>Due to this it gained a stigma being associated with poverty. The only people who ate it were slaves, indentured servants, and prisoners. A group of servants from Massachusetts actually sued because they were so sick of eating lobster and the courts ruled they would only have to eat it three times a week.</p>\n<p> </p>\n<p>It wasnt until the 1800s this started to change because railway managers discovered if they charged for lobster as if it were a delicacy to people who did not know its disgusting reputation then people did actually like it.  By 1920 

In [474]:
video = cv2.VideoWriter(videoName, 0, 32, (width, height))

# add in title in front
f = "/tmp/title-slide.png"
numFrames = framesPerCommentAt32fps[0]
for num in range(numFrames + 32):
    video.write(cv2.imread(f))
    
for i in range(len(frames)):
    f = frames[i][0]
    numFrames = framesPerCommentAt32fps[i+1]
    for num in range(numFrames):
        video.write(cv2.imread(f))
    if i < len(frames)-1:
        for j in range(0, 32):
            video.write(cv2.imread("/tmp/transition" + str(i) + "-" + str(i+1) + "f" + str(j) + ".png"))
    
# add outro slide in back
f = "/tmp/outro-slide.png"

numFrames = framesPerCommentAt32fps[len(framesPerCommentAt32fps) - 1]
for num in range(numFrames):
    video.write(cv2.imread(f))

cv2.destroyAllWindows()
video.release()

## Overlaying Audio onto Video

Finished currently and working for VLC, some encoding issue on Quicktime player

In [475]:
import moviepy.editor as mp

In [476]:
audio = mp.AudioFileClip("/tmp/combinedBacking.wav")
video = mp.VideoFileClip("/tmp/output.avi")
video.audio = audio

logo = (mp.ImageClip("logoWhite.png")
          .set_duration(video.duration)
          .resize(height=120) # if you need to resize...
          .margin(right=8, bottom=8, opacity=0) # (optional) logo-border padding
          .set_pos(("right","bottom")))

final = mp.CompositeVideoClip([video, logo])

final.write_videofile("outputSoundLogo.mp4")

chunk:   1%|          | 168/15342 [00:00<00:09, 1677.48it/s, now=None]

Moviepy - Building video outputSoundLogo.mp4.
MoviePy - Writing audio in %s


t:   0%|          | 11/22240 [00:00<03:23, 109.03it/s, now=None]        

MoviePy - Done.
Moviepy - Writing video outputSoundLogo.mp4



Moviepy - Done !
Moviepy - video ready outputSoundLogo.mp4


## Generate The Thumbnail

In [74]:
import nltk
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package punkt to /home/lukewulf/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/lukewulf/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [11]:
title = str(title)
capitalTitle = ""
for word in title.split(' '):
    capitalTitle = capitalTitle + word.capitalize() + ' '
tokens = nltk.word_tokenize(capitalTitle)

In [12]:
# JJ, NNS | NNP, NNS <- people in question
# JJ(S), NN, NN  <-subject of question
# VBN <- verb related to subject

highlightTokens = ['JJ', 'JJS', 'NN', 'NNP', 'NNS', 'VBN', 'VBG', 'VBD']
highlightWords = []
for pos in nltk.pos_tag(tokens):
    if pos[1] in highlightTokens:
        highlightWords.append(pos[0])


In [13]:
htmlTitle = capitalTitle
for word in highlightWords:
    htmlTitle = htmlTitle.replace(word, "<span class='highlight'>" + word + "</span>")

In [14]:
DRIVER = 'chromedriver'
driver = webdriver.Chrome(DRIVER)
driver.set_window_size(1600,1000)
url = "localhost:8000/thumbnail.html?image=" + str(1)\
        + "&background=" + urllib.quote_plus("#242424")\
        + "&highlight=" + urllib.quote_plus("#f29de1")\
        + "&silver=" + str(titleSilver)\
        + "&gold=" + str(titleGold)\
        + "&platinum=" + str(titlePlat)\
        + "&title=" + urllib.quote_plus(htmlTitle)

driver.get(url)
screenshot = driver.save_screenshot('/tmp/thumbnail.png')
driver.quit()